# Définition du modèle

Entrainement de la couche de classification seulement.

In [1]:
from sentence_transformers import SentenceTransformer
import torch
import torch.nn as nn

# Charger le modèle pré-entraîné
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Geler les poids pour ne pas réentraîner le modèle de base
for param in model.parameters():
    param.requires_grad = False

# Modèle de classification avec une couche fully-connected
class SentenceClassificationModel(nn.Module):
    def __init__(self, base_model):
        super(SentenceClassificationModel, self).__init__()
        self.base_model = base_model
        self.fc = nn.Linear(384, 2)  # Couche pour la classification binaire

    def forward(self, texts):
        # Encoder les textes en embeddings
        embeddings = self.base_model.encode(texts, convert_to_tensor=True).to(device)
        # Appliquer la couche fully-connected
        return self.fc(embeddings)

# Initialiser le modèle
custom_model = SentenceClassificationModel(model)

# Passer sur GPU si disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
custom_model = custom_model.to(device)


/home/onyxia/work/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-10-22 14:37:25.758810: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-22 14:37:25.778205: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-22 14:37:25.800858: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already b

# Définition du dataset

In [2]:
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset

# Charger le dataset Allociné
dataset = load_dataset("tblard/allocine")

# Redéfinir les ensembles train, validation, test
dataset = {
    'train': dataset['train'],
    'validation': dataset['validation'],
    'test': dataset['test']
}

# Classe Dataset pour PyTorch
class AllocineDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        # Récupérer texte et label
        text = self.dataset[idx]['review']
        label = self.dataset[idx]['label']
        return text, torch.tensor(label, dtype=torch.long)

# Créer les datasets
train_dataset = AllocineDataset(dataset['train'])
val_dataset = AllocineDataset(dataset['validation'])
test_dataset = AllocineDataset(dataset['test'])

# Afficher la taille des datasets
print(f"Taille train : {len(train_dataset)}")
print(f"Taille validation : {len(val_dataset)}")
print(f"Taille test : {len(test_dataset)}")

# Créer les DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

Taille train : 160000
Taille validation : 20000
Taille test : 20000


# Boucle d'entrainement et validation

In [3]:
from tqdm import tqdm
import torch.optim as optim

# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(custom_model.parameters(), lr=1e-3)

# Fonction d'entraînement du modèle avec visualisation et validation
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        # Mode entraînement
        model.train()
        running_loss = 0.0
        print(f"Epoch {epoch+1}/{num_epochs}:")

        # Boucle d'entraînement
        for inputs, labels in tqdm(train_loader, desc="Training", leave=False):
            # Les labels doivent être déplacés sur le GPU (si possible)
            labels = labels.to(device)
            # Réinitialiser les gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            
            # Calculer la perte
            loss = criterion(outputs, labels)
            
            # Backward pass et optimisation
            loss.backward()
            optimizer.step()

            # Accumuler la perte
            running_loss += loss.item()

        # Afficher la perte moyenne pour l'époque
        avg_train_loss = running_loss / len(train_loader)
        print(f'Epoch {epoch+1}/{num_epochs} completed. Training Loss: {avg_train_loss:.4f}')

        # Validation après chaque époque
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc="Validating", leave=False):
                # Les labels doivent être déplacés sur le GPU (si possible)
                labels = labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        avg_val_loss = val_loss / len(val_loader)
        accuracy = 100 * correct / total
        print(f'Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.2f}%\n')

In [4]:
# Fonction d'évaluation du modèle avec visualisation
def evaluate(model, test_loader):
    model.eval()  # Mettre le modèle en mode évaluation
    correct = 0
    total = 0
    print("Evaluating model on test data...")
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Evaluating", leave=False):
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on test data: {accuracy:.2f}%\n')
    return accuracy

# Entrainement

In [5]:
# Entraîner le modèle
print("Starting training...")
train(custom_model, train_loader, val_loader, criterion, optimizer, num_epochs=3)

# Évaluer le modèle sur les données de test
print("Training complete. Starting evaluation...")
evaluate(custom_model, test_loader)

Starting training...
Epoch 1/3:


Epoch 1/3 completed. Training Loss: 0.5752


Validation Loss: 0.5325, Accuracy: 73.86%

Epoch 2/3:


Epoch 2/3 completed. Training Loss: 0.5320


Validation Loss: 0.5170, Accuracy: 74.70%

Epoch 3/3:


Epoch 3/3 completed. Training Loss: 0.5223


Validation Loss: 0.5112, Accuracy: 74.79%

Training complete. Starting evaluation...
Evaluating model on test data...


Accuracy on test data: 74.62%



74.62

Nouvel entrainement en dégelant les deux dernières couches d'encoder du modèle de Sentence-Transformers.

In [6]:
from sentence_transformers import SentenceTransformer
import torch
import torch.nn as nn

# Charger le modèle SentenceTransformer pré-entraîné
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Geler tous les paramètres au départ
for param in model.parameters():
    param.requires_grad = False

# Identifier les couches BERT du modèle
# Ces couches peuvent être accessibles en accédant au modèle sous-jacent 'auto_model'
# Pour BERT, les couches sont généralement dans l'attribut 'encoder.layer'
bert_model = model[0].auto_model

# Dégeler les dernières 2 couches
for layer in bert_model.encoder.layer[-2:]:
    for param in layer.parameters():
        param.requires_grad = True

# Créer la couche de classification
class SentenceClassificationModel(nn.Module):
    def __init__(self, base_model, embedding_dim):
        super(SentenceClassificationModel, self).__init__()
        self.base_model = base_model
        self.fc = nn.Linear(embedding_dim, 2)

    def forward(self, texts):
        embeddings = self.base_model.encode(texts, convert_to_tensor=True).to(device)
        x = self.fc(embeddings)
        return x

# Obtenir la dimension de l'embedding
embedding_dim = model.get_sentence_embedding_dimension()

# Initialiser le modèle de classification personnalisé
custom_model = SentenceClassificationModel(model, embedding_dim)

# Passer le modèle sur GPU (si disponible)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
custom_model = custom_model.to(device)


In [7]:
# Entraîner le modèle
print("Starting training...")
train(custom_model, train_loader, val_loader, criterion, optimizer, num_epochs=3)

# Évaluer le modèle sur les données de test
print("Training complete. Starting evaluation...")
evaluate(custom_model, test_loader)

Starting training...
Epoch 1/3:


Epoch 1/3 completed. Training Loss: 0.6954


Validation Loss: 0.6946, Accuracy: 49.80%

Epoch 2/3:


Epoch 2/3 completed. Training Loss: 0.6954


Validation Loss: 0.6946, Accuracy: 49.80%

Epoch 3/3:


Epoch 3/3 completed. Training Loss: 0.6954


Validation Loss: 0.6946, Accuracy: 49.80%

Training complete. Starting evaluation...
Evaluating model on test data...


Accuracy on test data: 51.05%



51.055

Les dernières couches ne produisent pas de résultats probants, car elles capturent des relations plus subtiles entre les mots dans une langue. Étant donné que le modèle a été entraîné en anglais, et que l'objectif est de l'adapter à des données en français, il serait plus judicieux de dégeler les premières couches. Celles-ci se concentrent davantage sur la compréhension de la syntaxe et des structures de base, permettant ainsi au modèle de mieux s’adapter aux caractéristiques linguistiques fondamentales du français.